In [28]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import import_ipynb
from data_prep import data_pipeline

In [29]:
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.18.0


In [30]:
data_filepath = "./data/data.csv"

In [31]:
df = pd.read_csv(data_filepath)

In [32]:
X_train, X_test, y_train, y_test = data_pipeline(df)

In [33]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_dim=4, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100


/home/itachi/.pyenv/versions/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-18 17:16:00.672615: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


38438/38438 ━━━━━━━━━━━━━━━━━━━━ 62s 2ms/step - loss: 15133.1523 - mae: 76.9471
Epoch 2/100
38438/38438 ━━━━━━━━━━━━━━━━━━━━ 68s 2ms/step - loss: 3789.6094 - mae: 52.1329
Epoch 3/100
 9640/38438 ━━━━━━━━━━━━━━━━━━━━ 57s 2ms/step - loss: 3793.2856 - mae: 52.1730

KeyboardInterrupt: 